In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from functools import reduce
import operator
from collections import namedtuple
from operator import attrgetter

In [14]:
posts = pd.read_csv('data/instagram_posts.csv', sep='\t', nrows=100000, usecols=['profile_id', 'cts'], parse_dates=['cts'])

In [76]:
d = '2017-08-06T20:06:57.000000000'
d = d[:-3]
#datetime.fromisoformat(d)
datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f')
times = np.array([np.datetime64('2017-08-06T20:12:57.000000000'), np.datetime64('2017-08-06T20:06:57.000000000'), np.datetime64('2017-08-06T20:04:57.000000000')], dtype=np.datetime64)
#reduce(lambda a, b : a - b, times)
delta = times[0] - times[-1]
delta / 4
#max(np.datetime64('2017-08-06T20:12:57.000000000'), np.datetime64('2017-08-06T20:15:57.000000000'))

numpy.timedelta64(120000000000,'ns')

In [51]:
all_profile = pd.DataFrame(columns=['profile_id', 'min_post_time', 'max_post_time'])
len(all_profile)

0

In [54]:
all_times = {}
Profile = namedtuple('Profile', ['min_post_time', 'max_post_time', 'n_posts'])

all_profiles = {}

all_profile = pd.DataFrame(columns=['profile_id', 'min_post_time', 'max_post_time', 'n_posts'])
# I just need the min and max for each profile

for profile_id in posts.profile_id.unique():
    user_posts = posts[posts.profile_id == profile_id]
    current_min_post_time = user_posts.cts.min()
    current_max_post_time = user_posts.cts.max()
    
    """found_profile = all_profile[all_profile.profile_id == profile_id]
    if found_profile.empty:
        new_profile = pd.DataFrame(data=[profile_id, current_min_post_time, current_max_post_time])
        all_profile = pd.concat([all_profile, new_profile])
    else:
        previous_min_post_time = found_profile.min_post_time[0]
        previous_max_post_time = found_profile.max_post_time[0]
        found_profile.min_post_time[0] = current_min_post_time
        found_profile.max_post_time[0] = current_max_post_time
        found_profile.n_posts[0] += user_posts.count()
    """
    
    try:
        previous_min_post_time = all_profiles[profile_id].min_post_time 
        previous_max_post_time = all_profiles[profile_id].max_post_time 

        all_profiles[profile_id].min_post_time = min(previous_min_post_time, current_min_post_time)
        all_profiles[profile_id].max_post_time = max(previous_max_post_time, current_max_post_time)
        all_profiles[profile_id].n_posts += len(user_posts)
        
    except KeyError:
        all_profiles[profile_id] = Profile(current_min_post_time, current_max_post_time, len(user_posts))

In [55]:
all_profiles

{2237947779: Profile(min_post_time=Timestamp('2017-08-06 20:06:57'), max_post_time=Timestamp('2017-08-06 20:06:57'), n_posts=1),
 5579335020: Profile(min_post_time=Timestamp('2017-06-19 09:31:16'), max_post_time=Timestamp('2017-06-19 09:31:16'), n_posts=1),
 313429634: Profile(min_post_time=Timestamp('2017-03-11 20:05:03'), max_post_time=Timestamp('2017-03-11 20:05:03'), n_posts=1),
 1837592700: Profile(min_post_time=Timestamp('2016-09-13 16:27:16'), max_post_time=Timestamp('2016-09-13 16:27:16'), n_posts=1),
 1131527143: Profile(min_post_time=Timestamp('2015-10-18 10:19:27'), max_post_time=Timestamp('2015-10-18 10:19:27'), n_posts=1),
 16262389: Profile(min_post_time=Timestamp('2018-12-14 18:16:15'), max_post_time=Timestamp('2018-12-14 18:16:15'), n_posts=1),
 35673872: Profile(min_post_time=Timestamp('2018-08-09 17:53:46'), max_post_time=Timestamp('2018-08-09 17:53:46'), n_posts=1),
 840873396: Profile(min_post_time=Timestamp('2018-08-23 22:00:59'), max_post_time=Timestamp('2018-08-2

In [60]:
sorted(all_profiles.items(), key=lambda profile : profile[-1][-1], reverse=True)

[(195747313,
  Profile(min_post_time=Timestamp('2015-12-25 22:58:54'), max_post_time=Timestamp('2019-05-28 12:17:35'), n_posts=41)),
 (4048041553,
  Profile(min_post_time=Timestamp('2017-12-29 09:00:26'), max_post_time=Timestamp('2019-05-27 15:31:28'), n_posts=32)),
 (3137983884,
  Profile(min_post_time=Timestamp('2019-02-22 12:11:47'), max_post_time=Timestamp('2019-05-28 13:30:21'), n_posts=31)),
 (53113992,
  Profile(min_post_time=Timestamp('2018-04-18 22:54:09'), max_post_time=Timestamp('2019-05-22 15:01:19'), n_posts=30)),
 (5343183789,
  Profile(min_post_time=Timestamp('2018-05-05 21:50:11'), max_post_time=Timestamp('2019-05-08 22:45:11'), n_posts=30)),
 (9163884725,
  Profile(min_post_time=Timestamp('2019-01-10 12:16:00'), max_post_time=Timestamp('2019-05-28 16:48:15'), n_posts=30)),
 (3656684547,
  Profile(min_post_time=Timestamp('2018-11-08 10:05:46'), max_post_time=Timestamp('2019-05-04 19:53:12'), n_posts=29)),
 (276454303,
  Profile(min_post_time=Timestamp('2017-10-13 19:52:

In [86]:
profiles_with_diff = list(map(lambda profile: (profile[0], profile[-1].max_post_time - profile[-1].min_post_time, profile[-1].n_posts), all_profiles.items()))
profiles_with_rate = list(map(lambda profile: (profile[0], profile[1] / profile[2]), profiles_with_diff))
# filter out all profiles that have a diff of zero because they only posted once

#print(profiles_with_diff)
#sorted(profiles_with_diff.items(), key=lambda profile : profile[-1][-1])
print(sorted(profiles_with_diff, key=lambda profile : profile[-1], reverse=True)[0])
print(sorted(profiles_with_rate, key=lambda profile : profile[-1], reverse=False)[0])
print(list(filter(lambda profile : profile[0] == 7287439, profiles_with_diff)))
#print(profiles_with_rate[])

(195747313, Timedelta('1249 days 13:18:41'), 41)
(2237947779, Timedelta('0 days 00:00:00'))
[(7287439, Timedelta('1691 days 19:08:10'), 2)]


In [34]:
data = ['2017-08-06T20:06:57.000000000', '2017-06-19T09:31:16.000000000']
test_df = pd.DataFrame(data, columns=["cts"])
test_df = pd.to_datetime(test_df.cts)

In [35]:
test_df.cts.diff()

AttributeError: 'Series' object has no attribute 'cts'